In [2]:
!curl https://raw.githubusercontent.com/vseloved/prj-nlp-2019/master/tasks/07-language-as-sequence/run-on-test.json --output run-on-test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  273k  100  273k    0     0   492k      0 --:--:-- --:--:-- --:--:--  492k


In [5]:
import json

with open('run-on-test.json') as f:
    run_on_js = json.load(f)

In [6]:
import random

def rand_run_on_num():
    x = random.random()
    if x < 0.25:
        return 0
    elif 0.25 <= x < 0.97:
        return 1
    
    return 2

In [553]:
from nltk.corpus import brown
from nltk.corpus import reuters
from nltk.corpus import gutenberg
from nltk.corpus import webtext

source = []

for fileid in reuters.fileids():
    source.append(reuters.sents(fileid))
    
#for fileid in webtext.fileids():
#    source.append(webtext.sents(fileid))
    
for fileid in gutenberg.fileids():
    source.append(gutenberg.sents(fileid))    
    
for fileid in brown.fileids():
    source.append(brown.sents(fileid))    


In [2]:
from collections import defaultdict
import nltk

In [554]:
import spacy

nlp = spacy.load('en_core_web_lg', disable = 'ner')

In [ ]:
from collections import namedtuple

Token = namedtuple('Token', 'text point_after pos tag lemma dep')

random.seed(273757)

bigrams = defaultdict(float)
trigrams = defaultdict(float)
corpus = []

def tag_tokens(tokens):
    
    #tagged = [[token, False] for token in tokens]
    
    #if tagged[-1][0][-1] in '.!?':
    #    tagged[-1] = ['.', False]
    #else:
    #    tagged.append(['.', False])        
    
    doc = nlp(' '.join(tokens))
    
    return [Token(token.text, False, token.pos_, token.tag_, token.lemma_, token.dep_) for token in doc]

with open('run-on-corpus.txt', 'w') as f_corpus:
    for sents in source:        
        run_on_num = rand_run_on_num()
        buff = []
        prev_offset = 0
        file_tokens = []
        for sent in sents[1:]:
            if len(sent) < 2:
                continue
                
            tokens = tag_tokens(sent)
            buff.extend(tokens)
            file_tokens.extend(map(lambda t: t.text.lower(),tokens))
                
            if prev_offset > 0:                
                if buff[prev_offset - 1].text == '.':
                    del buff[prev_offset - 1]
                    buff[prev_offset - 2] = buff[prev_offset - 2]._replace(point_after = True)                
                   
                    if not buff[prev_offset - 1].text.isupper():                
                        lwr = random.random()                
                        if lwr < 0.93:
                            buff[prev_offset - 1] = buff[prev_offset -1]._replace(text = buff[prev_offset - 1].text.lower())                                            
            
            prev_offset = len(buff)            
            
            if run_on_num == 0:
                print(buff, file = f_corpus, end = "\n")
                corpus.append(buff)
                run_on_num = rand_run_on_num()
                prev_offset = 0
                buff = []                               
                       
            run_on_num -= 1
     
        for g in nltk.ngrams(file_tokens, 2):        
            bigrams[g] += 1.0
                
        for g in nltk.ngrams(file_tokens, 3):    
            trigrams[g] += 1.0    

In [552]:
len(corpus)

16136

In [9]:
def update_freq(ngrams):
    n = sum(ngrams.values())
    for k, v in ngrams.items():
        ngrams[k] = v/n

In [531]:
update_freq(bigrams)
update_freq(trigrams)

In [11]:
sorted(bigrams.items(), key=lambda kv: kv[1], reverse = True)

[(("'", 's'), 0.006238248091994165),
 (('.', 'the'), 0.005556772806303563),
 (('said', '.'), 0.005003594823786987),
 (('of', 'the'), 0.004939446172199974),
 (('in', 'the'), 0.0047036790501334205),
 (('.', '"'), 0.003992212187077462),
 (('u', '.'), 0.003435701807075846),
 ((',', '"'), 0.003219929069919531),
 (('.', 's'), 0.0031991015856380335),
 (('s', '.'), 0.003123289542853382),
 ((',', 'the'), 0.0029799964509966783),
 ((',', '000'), 0.002826706166684856),
 (('said', 'the'), 0.0028233737691998164),
 (('mln', 'dlrs'), 0.002767556111325403),
 (('the', 'company'), 0.0023518395250667103),
 (('he', 'said'), 0.002079416030664722),
 (('1', '.'), 0.0020452589564430657),
 (('for', 'the'), 0.0019261257463528993),
 (('to', 'the'), 0.0018211552255741512),
 (('said', 'it'), 0.0016178789789867345),
 ((',', 'which'), 0.0015329028431182244),
 (('on', 'the'), 0.0014437612103934145),
 (('the', 'u'), 0.001439595713537115),
 ((',', 'and'), 0.0014187682292556175),
 (('it', 'said'), 0.0013221287021894684),

In [535]:
train_index = int(0.7 * len(corpus))

train_data = corpus[: train_index]
test_data = corpus[train_index: ]

In [536]:
import math

def baseline(data):
    result_data = []
    for sent in data:
        result_sent = []
        last_point = 0
        for i, word in enumerate(sent):            
            if (i - last_point) > 3 and i < (len(sent) - 1):
                pbigram = (word[0].lower(), '.')                
                bigram = (word[0].lower(), sent[i+1][0].lower())                                    
                if (sent[i + 1][0][0:1].isupper()):
                    result_sent.append([word[0], True])
                elif math.log(bigrams[pbigram] + 0.000000000001) > math.log(bigrams[bigram] + 0.001):    
                    #if not word[1]:
                         #print('false positive', pbigram, bigram, bigrams[pbigram], bigrams[bigram], word, sent[i + 1])
                    result_sent.append([word[0], True])
                    last_point = i
                else:   
                    #if word[1]:
                    #     print('false negative', pbigram, bigram, bigrams[pbigram], bigrams[bigram], word, sent[i + 1])
                    result_sent.append([word[0], False])                                    
            else:
                #if word[1]:
                #    print('false negative', word, sent[i + 1])
                result_sent.append([word[0], False])
            
        result_data.append(result_sent)   
    
    return result_data

In [537]:
result = baseline(test_data)

In [538]:
def labels_vec(data):
    return [word[1] for sent in data for word in sent]

In [539]:
from sklearn.metrics import classification_report


print(classification_report(labels_vec(test_data), labels_vec(result)))

              precision    recall  f1-score   support

       False       0.99      0.91      0.95    172582
        True       0.02      0.20      0.03      1393

   micro avg       0.90      0.90      0.90    173975
   macro avg       0.51      0.55      0.49    173975
weighted avg       0.99      0.90      0.94    173975



In [540]:
print(classification_report(labels_vec(run_on_js), labels_vec(baseline(run_on_js))))

              precision    recall  f1-score   support

       False       0.98      0.96      0.97      4542
        True       0.26      0.45      0.33       155

   micro avg       0.94      0.94      0.94      4697
   macro avg       0.62      0.70      0.65      4697
weighted avg       0.96      0.94      0.95      4697



In [541]:
import sys

def extract_sent_features(sent):
    features = []
    for i, token in enumerate(sent):
        fdic = {}        
        fdic['next_case'] = sent[i+1].text[:1].isupper() if i < (len(sent) - 1) else False        
        fdic['case'] = token.text[:1].isupper()
        
        fdic['tag'] = token.tag
        fdic['tag_prev'] = str(None) if i == 0 else str(sent[i-1].tag)
        fdic['tag_next'] = str(None) if i == (len(sent) - 1) else str(sent[i+1].tag)        
        
        fdic['index'] = len(sent)/(i+1)
        
        
        fdic['word'] = token.text
        fdic['prev_word'] = str(None) if i == 0 else sent[i-1].text
        fdic['next_word'] = str(None) if i == (len(sent) - 1) else sent[i+1].text        
        
        fdic['bigram'] = math.log((0.0 if i == (len(sent) - 1) else bigrams[(token.text.lower(), sent[i+1].text.lower())]) +  sys.float_info.epsilon)
        fdic['pbigram'] = math.log((0.0 if i == (len(sent) - 1) else bigrams[(token.text.lower(), '.')]) +  sys.float_info.epsilon)
        fdic['pbigram2'] = math.log((0.0 if i == (len(sent) - 1) else bigrams[('.', sent[i+1].text.lower())]) +  sys.float_info.epsilon)      
        fdic['ptrigram'] = math.log((0.0 if i == (len(sent) - 1) else trigrams[(token.text.lower(), '.', sent[i+1].text.lower())]) +  sys.float_info.epsilon)
        fdic['ptrigram2'] = math.log((0.0 if i > (len(sent) - 3) else trigrams[('.', sent[i+1].text.lower(), sent[i+2].text.lower())]) +  sys.float_info.epsilon)
        fdic['ptrigram3'] = math.log((trigrams[(sent[i-1].text.lower(), token.text.lower(), '.')] if i > 0 else 0.0) +  sys.float_info.epsilon)
        
        
        fdic['dep'] = token.dep
        fdic['prev_dep'] = str(None) if i == 0 else sent[i-1].dep
        fdic['next_dep'] = str(None) if i == (len(sent) - 1) else sent[i+1].dep       
        
        features.append(fdic)
    
    return features

def extract_features(sents):
    features = []
    for sent in sents:
        features.extend(extract_sent_features(sent))
    return features 
    

In [542]:
features = extract_features(train_data)

In [543]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()

vectorizer.fit(features)

feature_vecs = vectorizer.transform(features)

In [544]:
from sklearn.linear_model import LogisticRegression 

#logreg = LogisticRegression(random_state=26,  solver='lbfgs', multi_class="multinomial", max_iter=2000)
logreg = LogisticRegression(random_state=26, max_iter=3000)

logreg.fit(feature_vecs, labels_vec(train_data))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=3000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=26, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [545]:
def predict(data):
    vec = vectorizer.transform(extract_features(data))
    return logreg.predict(vec)

In [546]:
test_predicted = predict(test_data)

In [547]:
print(classification_report(labels_vec(test_data), test_predicted))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    172582
        True       0.93      0.86      0.89      1393

   micro avg       1.00      1.00      1.00    173975
   macro avg       0.96      0.93      0.95    173975
weighted avg       1.00      1.00      1.00    173975



In [548]:
def convert_to_tokens(data):
    tagged_sents = []
    
    for sent in data:
        tagged_sent = [] 
        tokens = tag_tokens(list(map(lambda w: w[0], sent)))
        compound = None
        merged = 0
        i = 0
        j = 0
        while i < len(tokens) or j < len(sent): 
            orig_token = sent[i] if i < len(sent) else sent[len(sent) - 1]
            token = tokens[j] if j < len(tokens) else tokens[len(tokens) - 1]           
            if orig_token[0] == token.text:                
                tagged_sent.append(token._replace(point_after = orig_token[1]))
                j += 1
                i += 1
            elif orig_token[0] in token.text:                
                if not compound:
                    compound = token
                else:
                    compound = compound._replace(point_after = orig_token[1])
                merged += len(orig_token[0])    
                if merged == len(compound.text):                     
                    j += 1
                    tagged_sent.append(compound)
                    compound = None                
                    merged = 0
                i += 1    
            elif token.text in orig_token[0]:                
                if not compound:
                    compound = orig_token[0]
                    
                merged += len(token.text)
                if merged == len(compound): 
                    i += 1
                    tagged_sent.append(token._replace(point_after = orig_token[1]))                    
                    compound = None
                    merged = 0
                else:
                    tagged_sent.append(token)
                j += 1
            else:                
                print(orig_token, token.text)
                print(list(map(lambda w: w[0], tagged_sent)))
                print(list(map(lambda w: w[0], sent)))
                assert False                                    
        
        tagged_sents.append(tagged_sent)
    
    return tagged_sents

run_on_js_tokenized = convert_to_tokens(run_on_js)

In [549]:
print(sum([w[1] for sent in run_on_js for w in sent]))
print(sum([t.point_after for sent in run_on_js_tokenized for t in sent]))

155
155


In [550]:
run_on_js_tokenized[1]

[Token(text='Economists', point_after=False, pos='NOUN', tag='NNS', lemma='economist', dep='nsubj'),
 Token(text='on', point_after=False, pos='ADP', tag='IN', lemma='on', dep='prep'),
 Token(text='both', point_after=False, pos='DET', tag='DT', lemma='both', dep='preconj'),
 Token(text='the', point_after=False, pos='DET', tag='DT', lemma='the', dep='det'),
 Token(text='left', point_after=False, pos='ADJ', tag='JJ', lemma='left', dep='pobj'),
 Token(text='and', point_after=False, pos='CCONJ', tag='CC', lemma='and', dep='cc'),
 Token(text='right', point_after=False, pos='ADV', tag='RB', lemma='right', dep='advmod'),
 Token(text='broadly', point_after=False, pos='ADV', tag='RB', lemma='broadly', dep='advmod'),
 Token(text='agree', point_after=False, pos='VERB', tag='VBP', lemma='agree', dep='ROOT'),
 Token(text='that', point_after=False, pos='ADP', tag='IN', lemma='that', dep='mark'),
 Token(text='the', point_after=False, pos='DET', tag='DT', lemma='the', dep='det'),
 Token(text='need', po

In [551]:
print(classification_report(labels_vec(run_on_js_tokenized), predict(run_on_js_tokenized)))

              precision    recall  f1-score   support

       False       0.98      0.99      0.98      4618
        True       0.47      0.37      0.41       155

   micro avg       0.97      0.97      0.97      4773
   macro avg       0.72      0.68      0.70      4773
weighted avg       0.96      0.97      0.96      4773

